In [12]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [13]:

MODEL_PATH = r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\plank_model\model\plank_mlp_full.pt"
SCALER_PATH = r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\plank_model\model\scaler_full.pkl"


IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER", "RIGHT_SHOULDER",
    "LEFT_ELBOW", "RIGHT_ELBOW",
    "LEFT_WRIST", "RIGHT_WRIST",
    "LEFT_HIP", "RIGHT_HIP",
    "LEFT_KNEE", "RIGHT_KNEE",
    "LEFT_ANKLE", "RIGHT_ANKLE",
    "LEFT_HEEL", "RIGHT_HEEL",
    "LEFT_FOOT_INDEX", "RIGHT_FOOT_INDEX",
]

In [16]:
with open(SCALER_PATH, "rb") as f:
    scaler = pickle.load(f)


class MLP_Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
        return self.net(x)

#input_dim = len(IMPORTANT_LMS) * 4  # x,y,z,v
input_dim = 33 * 4
model = MLP_Model(input_dim)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

C:\Users\caovi\AppData\Local\Temp\ipykernel_15896\1668664326.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


MLP_Model(
  (net): Sequential(
    (0): Linear(in_features=132, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [17]:
#def extract_keypoints(results):
#    keypoints = []
#    lm = results.pose_landmarks.landmark

 #   for lm_name in IMPORTANT_LMS:
  #      p = lm[mp_pose.PoseLandmark[lm_name].value]
   #     keypoints.extend([p.x, p.y, p.z, p.visibility])

#    return np.array(keypoints).reshape(1, -1)

def extract_full_landmarks(results):
    lm = results.pose_landmarks.landmark
    row = []
    for p in lm:
        row.extend([p.x, p.y, p.z, p.visibility])
    return np.array(row).reshape(1, -1)


In [21]:

cap = cv2.VideoCapture(r"C:\Users\caovi\Downloads\IMG_5634.MOV")
threshold = 0.25  # confidence threshold
#r"C:\Users\caovi\Downloads\IMG_5630.mp4"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:

            mp_drawing.draw_landmarks(
                img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            #row = extract_keypoints(results)
            row = extract_full_landmarks(results)

            row_scaled = scaler.transform(row)
            row_tensor = torch.tensor(row_scaled, dtype=torch.float32)

            with torch.no_grad():
                prob = torch.sigmoid(model(row_tensor)).item()

            # Classification
            if prob < threshold:
                pred = "BAD FORM"
                color = (0,0,255)
            else:
                pred = "GOOD FORM"
                color = (0,255,0)

            # Display
            cv2.putText(img, f"{pred} ({prob:.2f})",
                        (10,40), cv2.FONT_HERSHEY_SIMPLEX,
                        1, color, 2)

        cv2.imshow("Plank Detection - MLP", img)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

for i in range(5):
    cv2.waitKey(1)

C:\Users\caovi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\caovi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\caovi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\caovi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\caovi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but St